In [140]:
from pathlib import Path
from collections import deque, defaultdict

In [143]:
test_input = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

In [214]:
FILESYSTEM_SIZE = 70000000
UPDATE_SIZE = 30000000


class Directory:
    def __init__(self, path, parent):
        self._path = path
        self._files = {}
        self._directories = {}
        self.parent = parent
        
    def ls(self, listing):
        for info, name in listing:
            if info == "dir":
                self._directories[name] = Directory(f"{self._path}{name}/", self)
            else:
                self._files[name] = int(info)
    
    def cd(self, directory):
        if directory == "..":
            return self.parent
        return self._directories[directory]
                
    def local_size(self) -> int:
        return sum(filesize for filesize in self._files.values())

    def size(self) -> int:
        return self.local_size() + sum(directory.size() for directory in self._directories.values())
    
    def all_directories(self):
        return [self] + [subdir for directory in self._directories.values() for subdir in directory.all_directories()]

    
class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size


def parse_terminal(output):
    global ALL_FILES
    ALL_FILES = {}
    return deque([tuple(row.split()) for row in output.strip().split("\n")])


def map_filesystem(terminal_output: deque):
    filesystem_root = Directory("/", None)
    cwd = None
    while terminal_output:
        command = terminal_output.popleft()
        match command:
            case "$", "cd", path:
                if path == "/":
                    cwd = filesystem_root
                else:
                    cwd = cwd.cd(path)
            case "$", "ls":
                listing = []
                while terminal_output and "$" not in terminal_output[0]:
                    listing.append(terminal_output.popleft())
                cwd.ls(listing)
    return filesystem_root


filesystem_root = map_filesystem(parse_terminal(test_input))
assert filesystem_root.cd("a").cd("e").size() == 584
assert filesystem_root.cd("a").size() == 94853
assert filesystem_root.cd("d").size() == 24933642
assert filesystem_root.size() == 48381165
assert len(filesystem_root.all_directories()) == 4

In [215]:
# Part 1 - Test
filesystem_root = map_filesystem(parse_terminal(test_input))
assert sum(size for size in [d.size() for d in filesystem_root.all_directories()] if size <= 100000) == 95437

In [216]:
# Part 1
# Guesses: 1293614 (low)
output = parse_terminal(Path("input.txt").read_text())
filesystem_root = map_filesystem(output)
print(sum(size for size in [d.size() for d in filesystem_root.all_directories()] if size <= 100000))

1367870


In [217]:
# Part 2 - Test
filesystem_root = map_filesystem(parse_terminal(test_input))
available_space = FILESYSTEM_SIZE - filesystem_root.size()
needed_space = UPDATE_SIZE - available_space
smallest_enough_directory_size = sorted([size for size in [d.size() for d in filesystem_root.all_directories()] if size >= needed_space])[0]
assert smallest_enough_directory_size == 24933642

In [218]:
# Part 2
filesystem_root = map_filesystem(parse_terminal(Path("input.txt").read_text()))
available_space = FILESYSTEM_SIZE - filesystem_root.size()
needed_space = UPDATE_SIZE - available_space
smallest_enough_directory_size = sorted([size for size in [d.size() for d in filesystem_root.all_directories()] if size >= needed_space])[0]
print(smallest_enough_directory_size)

549173
